In [ ]:
import numpy as np

# Load the NumPy file with allow_pickle=True
data = np.load('data.npy', allow_pickle=True)

# View the contents
print(data)


In [ ]:
import pandas as pd
df = pd.DataFrame(data)

df

In [ ]:
dict = {}
for ind in df.index:
    # print (df[3][ind])
    if(dict.get(df[3][ind]) != None):
        dict[df[3][ind]]+= 1
    else:
        dict[df[3][ind]]= 1

print(dict)


In [ ]:
import matplotlib.pyplot as plt

for key in dict:
    plt.bar(key, dict[key])

plt.show()


<h3> KNN FROM SCRATCH </h3>

In [ ]:
np.random.seed(100)
np.random.shuffle(data)


splitind = int(len(data) * 4/5)

trainarr = data[:splitind]
testarr = data[splitind:]

# print(trainarr[3])
trainlabels = []
testlabels = []
for i in trainarr:
    trainlabels.append(i[3])
for i in testarr:
    testlabels.append(i[3])
print(len(trainlabels))

<h3> euclidean distance </h3>

In [ ]:
predlabels =  []
for testrec in testarr:
    kneighbours = {} # Initialize as an empty list
    count = 0

    for trainrec in trainarr:
        # for resnet
        diff = np.subtract(testrec[2], trainrec[2])
        sq = np.square(diff)
        axissum = np.sum(sq)
        eucdist = np.sqrt(axissum)  # euc distance btw given test pt and train pt

        kneighbours[count] = eucdist
        count+=1
    
    labelsdict = {}
    sorted_distances = sorted(kneighbours.items(), key=lambda x: x[1])

    for i in range(0, 5):
        index = sorted_distances[i][0]

        if labelsdict.get(trainlabels[index]) is not None:
            labelsdict[trainlabels[index]] += 1
        else:
            labelsdict[trainlabels[index]] = 1

    sorted_labels = sorted(labelsdict.items(), key=lambda x: x[1], reverse=True)
    # break
    if len(sorted_labels) >= 2:
        if sorted_labels[0][1] == sorted_labels[1][1]:
            predlabels.append(trainlabels[sorted_distances[0][0]])
        else:
            predlabels.append(sorted_labels[0][0])
    else:
        # Handle the case where there are fewer than two elements in sorted_labels
        predlabels.append(sorted_labels[0][0] if sorted_labels else None)


<h3> manhattan distance </h3>

In [ ]:
predlabels =  []
for testrec in testarr:
    kneighbours = {} # Initialize as an empty list
    count = 0

    for trainrec in trainarr:
        # for resnet
        diff = np.subtract(testrec[2], trainrec[2])
        abso = np.abs(diff)
        # axissum = 
        mandist = np.sum(abso) # euc distance btw given test pt and train pt

        kneighbours[count] = mandist
        count+=1
    
    labelsdict = {}
    sorted_distances = sorted(kneighbours.items(), key=lambda x: x[1])

    for i in range(0, 5):
        index = sorted_distances[i][0]

        if labelsdict.get(trainlabels[index]) is not None:
            labelsdict[trainlabels[index]] += 1
        else:
            labelsdict[trainlabels[index]] = 1

    sorted_labels = sorted(labelsdict.items(), key=lambda x: x[1], reverse=True)
    # break
    if len(sorted_labels) >= 2:
        if sorted_labels[0][1] == sorted_labels[1][1]:
            predlabels.append(trainlabels[sorted_distances[0][0]])
        else:
            predlabels.append(sorted_labels[0][0])
    else:
        # Handle the case where there are fewer than two elements in sorted_labels
        predlabels.append(sorted_labels[0][0] if sorted_labels else None)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(testlabels, predlabels)
precision = precision_score(testlabels, predlabels, average="macro")
recall = recall_score(testlabels, predlabels, average="macro")
f1 = f1_score(testlabels, predlabels, average="macro")

print(accuracy)
print(precision)
print(recall)
print(f1)


<H3> KNN CLASS </H3>

In [ ]:
class Knn():
    def __init__(self, k, encoder_name, distName, trainarr, testarr):
        self.k = k
        self.encoder = encoder_name
        self.distMetric = distName
        self.trainlabels = []
        self.testlabels = []
        self.predictedlabels = []
        for i in trainarr:
            self.trainlabels.append(i[3])
        for i in testarr:
            self.testlabels.append(i[3])
            
    def calcdist(self, test, train):
        if self.distMetric == "euclidean distance":
            diff = np.subtract(test, train)
            sq = np.square(diff)
            axissum = np.sum(sq)
            eucdist = np.sqrt(axissum)
            return eucdist
        
        elif self.distMetric == "manhattan distance":
            diff = np.subtract(test, train)
            abso = np.abs(diff)
            mandist = np.sum(abso)
            return mandist
        # elif self.distMetric == "cosine distance":
        else:
            print("invalid distance type")
            return
            
    def predict(self):
        predlabels =  []
        for testrec in testarr:
            kneighbours = {} # Initialize as an empty list
            count = 0

            for trainrec in trainarr:
                # for resnet
                if(self.encoder == "resnet"):
                    dist = self.calcdist(testrec[1], trainrec[1])
                elif(self.encoder == "vit"):
                    dist = self.calcdist(testrec[2], trainrec[2])
                else:
                    print("invalid encoder type")
                    return
                
                kneighbours[count] = dist
                count+=1
            
            labelsdict = {}
            sorted_distances = sorted(kneighbours.items(), key=lambda x: x[1])

            for i in range(0, self.k):
                index = sorted_distances[i][0]

                if labelsdict.get(trainlabels[index]) is not None:
                    labelsdict[trainlabels[index]] += 1
                else:
                    labelsdict[trainlabels[index]] = 1

            sorted_labels = sorted(labelsdict.items(), key=lambda x: x[1], reverse=True)
            # break
            if len(sorted_labels) >= 2:
                if sorted_labels[0][1] == sorted_labels[1][1]:
                    predlabels.append(trainlabels[sorted_distances[0][0]])
                else:
                    predlabels.append(sorted_labels[0][0])
            else:
                # Handle the case where there are fewer than two elements in sorted_labels
                predlabels.append(sorted_labels[0][0] if sorted_labels else None)
            # print("hi")
        self.predictedlabels = predlabels
        return 
    
    def metrics(self):
        # self.predict()
        accuracy = accuracy_score(self.testlabels, self.predictedlabels)
        precision = precision_score(self.testlabels, self.predictedlabels, average="macro")
        recall = recall_score(self.testlabels, self.predictedlabels, average="macro")
        f1 = f1_score(self.testlabels, self.predictedlabels, average="macro")
        
        # print(accuracy)
        # print(precision)
        # print(recall)
        # print(f1)
        return accuracy

<H3> ACCURACY, PRECISION, RECALL, F1 SCORE </H3>

In [ ]:
knnobj = Knn(5, "vit", "manhattan distance", trainarr, testarr)
knnobj.predict()
knnobj.metrics()

<h2> BEST COMBINATION OF PARAMETERS </H2>

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

distance = ["manhattan distance", "euclidean distance"]
encoder = ["resnet", "vit"]
combinations = []
for k in range(1, 150):
    for enc in encoder:
        for dis in distance:
            knnInstance = Knn(k, enc, dis, trainarr, testarr)
            knnInstance.predict()
            acc = knnInstance.metrics()
            combinations.append([k, enc, dis, acc])

combinations.sort(key=lambda x: x[3], reverse=True)
for i in range(min(20, len(combinations))):
    print(f"Combination {i + 1}: k={combinations[i][0]}, encoder={combinations[i][1]}, distance={combinations[i][2]}, accuracy={combinations[i][3]}")


In [ ]:

distance = ["manhattan distance"]
encoder = ["resnet"]
# combinations = []
x = []
y = []
for k in range(1, 150):
    for enc in encoder:
        for dis in distance:
            knnInstance = Knn(k, enc, dis, trainarr, testarr)
            knnInstance.predict()
            acc = knnInstance.metrics()
            x.append(k)
            y.append(acc)
            print("k = ", k, end = "\t")
            print("accuracy = ", acc)


In [ ]:
import matplotlib.pyplot as plt
plt.xlabel("k value")
plt.ylabel("accuracy")
plt.title("k vs accuracy for RESNET")
plt.plot(x,y)
plt.show()